In [11]:
import numpy as np
import random

Write a simple Hamming encoder program in Python, which, when given a 4-bit binary value, returns the resulting 7-bit binary vector codeword. 
Also implement the parity check functionality to see if there are any errors, that is to check whether H ∗ cw = 0 holds, where is zero vector.

In [12]:
GeneratorMatrix = np.array([[1, 1, 0, 1],
                         [1, 0, 1, 1],
                         [1, 0, 0, 0],
                         [0, 1, 1, 1],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0],
                         [0, 0, 0, 1]])

ParityCheckMatrix = np.array([[1, 0, 1, 0, 1, 0, 1],
                             [0, 1, 1, 0, 0, 1, 1],
                             [0, 0, 0, 1, 1, 1, 1]])

DecoderMatrix = np.array([[0, 0, 1, 0, 0, 0, 0],
                         [0, 0, 0, 0, 1, 0, 0],
                         [0, 0, 0, 0, 0, 1, 0],
                         [0, 0, 0, 0, 0, 0, 1]])

In [17]:
#OLD CODE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Function takes 4-digit number and transforms number into an array consisting of those four elements
# Resulting array is used to calculate 7-digit codeword with the help of the Generator Matrix (G)

def codeword_generator(x):
    
    lst = [int(i) for i in str(x)]
    if len(lst) != 4:
        raise ValueError("Code must consist of four digits.")
    resulting_array = np.array(lst)
    
    preliminary_codeword = np.dot(GeneratorMatrix, resulting_array) % 2

    # GF(2) arithmetic: put a 1 in our 7-digit codeword, if number is odd and out a 0, if the number is even (e.g., if the resulting number is 2 due to 
    # two times 1, put a 0 to the resulting codeword, since it may only contain 0s and 1s.)
    codeword = np.array(preliminary_codeword)
    return codeword

# Multiplying the codeword with the Parity-check Matrix (H) must result in the zero Vector
# x is the codeword from the first function
# if codeword is artificially generated (with bitflip), x is the artificially generated codeword
def parity_check(x):

    # create array containing of 0s and 1s
    error_syndrome = np.dot(ParityCheckMatrix, x) % 2
    print(f'The error vector is {error_syndrome}.')
    if np.sum(error_syndrome) == 0:
        print(f'There was no error occuring upon code-transmission.')
    else:
        transposed_parity = ParityCheckMatrix.transpose()
        list_parity = transposed_parity.tolist()
        list_index = list_parity.index(error_syndrome.tolist())
        matrix_row_index = list_index + 1
        #condition = transposed_parity == error_syndrome
        #matrix_row_index = np.where(condition)[0]
        print(f'There was an error eoccuring upon code-transmission.\nThe Bitflip occured at the {matrix_row_index}th position.')
    return

# decoder function returns 'original' 4-bit vector when given a 7-bit codeword
def decoder(x):

    if type(x) is not np.ndarray:
        x = np.array(x)

    if (x.shape[0] != 7) and (x.shape[1] != 1):
        raise ValueError ('Function can only decode vectors of shape [7, 1].')
    
    decoded = np.dot(DecoderMatrix, x)
    print(decoded)
    binary_decoded = np.mod(decoded, 2)
    print(binary_decoded)
    return (f'The original 4-bit vector is {binary_decoded}.')

Hamming Class or encoding, decoding, parity check

In [26]:
class Support():

    @staticmethod
    def check_input(x, input_length):
        # Check if x is an integer, a list, or a NumPy array
        if not isinstance(x, (int, list, np.ndarray, str)):
            raise TypeError("The input must be an integer, a list, a string, or a NumPy array.")

        # If x is an integer, convert it to a list of its digits
        if isinstance(x, int):
            x = [int(digit) for digit in str(x)]

        # Remove white spaces if x is a string
        if isinstance(x, str):
            x = x.replace(" ", "")
            if not x.isdigit():
                raise ValueError("String input must contain only digits.")
            # Convert string of digits to list of integers
            x = [int(digit) for digit in x]

        # Convert x to a NumPy array if it's not already
        if not isinstance(x, np.ndarray):
            try:
                x = np.array(x)
            except Exception as e:
                raise ValueError(f"Failed to convert input to a NumPy array: {e}")

        # Check if the array has the specified input length
        if x.size != input_length:
            raise ValueError(f"The input array must have a size of {input_length}.")

        # Check if the elements are integers
        if not np.issubdtype(x.dtype, np.integer):
            raise ValueError("The input array must contain only integers.")

        # Check if the elements are either 0 or 1
        if not np.all(np.isin(x, [0, 1])):
            raise ValueError("The input array must contain only 0s and 1s.")

        return x

In [27]:
# test the check_input function
print(Support().check_input([1, 0, 1, 1], 4))
print(Support().check_input(10011, 5))
print(Support().check_input(np.array([1, 0, 1, 1]), 4))
print(Support().check_input("1 0 1 1", 4))

[1 0 1 1]
[1 0 0 1 1]
[1 0 1 1]
[1 0 1 1]


In [21]:
class Hamming:
    """
    Class for encoding, decoding and checking Hamming codes.
    """

# Function takes 4-digit number and transforms number into an array consisting of those four elements
# Resulting array is used to calculate 7-digit codeword with the help of the Generator Matrix (G)
    @staticmethod
    def encoder(input):
    
        binary_list = [int(i) for i in str(input)]
        if len(binary_list) != 4:
            raise ValueError("Code must consist of four digits.")
        resulting_array = np.array(binary_list)
    
        codeword = np.matmul(GeneratorMatrix, resulting_array) % 2
        print(f'The 7-bit codeword is {codeword}.')

        # GF(2) arithmetic: put a 1 in our 7-digit codeword, if number is odd and out a 0, if the number is even (e.g., if the resulting number is 2 due to 
        # two times 1, put a 0 to the resulting codeword, since it may only contain 0s and 1s.)
        return codeword
    
    @staticmethod
    def parity_check(codeword):
        
        error_syndrome = np.matmul(ParityCheckMatrix, codeword) % 2
        print(f'The error vector is {error_syndrome}.')

        sum_error_syndrome = np.sum(error_syndrome)
        if sum_error_syndrome == 0:
            print('There was no error occuring upon code-transmission.')
        elif sum_error_syndrome == 1:
            print('The sum of the error syndrome elements is 1. There was an error occuring upon code-transmission in either one of the parity bits or there were two errors occuring in the data bits. Either way, the error cannot be corrected.')
        else: # sum_error_syndrome == 2 or 3
            # transpose the matrix to be able to compare our error syndrome with the rows of the transposed matrix
            transposed_parity = ParityCheckMatrix.transpose()
            # convert the matrix to a list to be able to use the index function
            list_parity = transposed_parity.tolist()
            list_parity_index = list_parity.index(error_syndrome.tolist())
            # add 1 to the index to get the correct row number
            bitflip_pos = list_parity_index + 1
            print(f'There was an error eoccuring upon code-transmission and the error can be corrected.\nThe Bitflip occured at the {bitflip_pos}th position.')
        return

    @staticmethod
    def decoder(codeword):

        # if type(codeword) is not np.ndarray:
        #     codeword = np.array(codeword)

        # if (codeword.shape[0] != 7) and (codeword.shape[1] != 1):
        #     raise ValueError ('Function can only decode vectors of shape [7, 1].')
    
        error_syndrome = np.matmul(ParityCheckMatrix, codeword) % 2
        sum_error_syndrome = np.sum(error_syndrome)

        if sum_error_syndrome == 0:
            print('There is no bitflip-error in the codeword to be corrected.')
            # decoder function returns 'original' 4-bit vector when given a 7-bit codeword
            binary_decoded = np.matmul(DecoderMatrix, codeword) % 2
            print(f'The original 4-bit vector is {binary_decoded}.')
        elif sum_error_syndrome == 1:
            print('The sum of the error syndrome elements is 1. There is an error in either one of the parity bits or there are two errors in the data bits. In these cases, the error cannot be decoded.')
        else: # sum_error_syndrome == 2 or 3
             # transpose the matrix to be able to compare our error syndrome with the rows of the transposed matrix
            transposed_parity = ParityCheckMatrix.transpose()
            # convert the matrix to a list to be able to use the index function
            list_parity = transposed_parity.tolist()
            list_parity_index = list_parity.index(error_syndrome.tolist())
            # add 1 to the index to get the correct row number
            bitflip_pos = list_parity_index + 1
            print(f'The bit at position {bitflip_pos} was flipped.')

            # flip the bit at the position of the bitflip
            if codeword[bitflip_pos - 1] == 0:
                codeword[bitflip_pos - 1] = 1
            else:
                codeword[bitflip_pos - 1] = 0
            print(f'The corrected codeword is {codeword}.')
            binary_decoded = np.matmul(DecoderMatrix, codeword) % 2
            print(f'The original 4-bit vector is {binary_decoded}.')
        return

In [27]:
example1 = Hamming()


codeword = example1.encoder(1010)
print(codeword)

The 7-bit codeword is [1 0 1 1 0 1 0].
[1 0 1 1 0 1 0]


In [28]:
example1.parity_check(codeword)


The error vector is [0 0 0].
There was no error occuring upon code-transmission.


In [29]:
example1.decoder(codeword)

There is no bitflip-error in the codeword to be corrected.
The original 4-bit vector is [1 0 1 0].


In [ ]:
example2 = 1001
print(codeword_generator(example2))
print(parity_check(codeword))
print(decoder(codeword))

[0 0 1 1 0 0 1]
The error vector is [0 0 0].
There was no error occuring upon code-transmission.
None
[1 0 0 1]
[1 0 0 1]
The original 4-bit vector is [1 0 0 1].


In [ ]:
example3 = 1011
print(codeword_generator(example3))
print(parity_check(codeword))
print(decoder(codeword))

[0 1 1 0 0 1 1]
The error vector is [0 0 0].
There was no error occuring upon code-transmission.
None
[1 0 1 1]
[1 0 1 1]
The original 4-bit vector is [1 0 1 1].


In [26]:
# Artificial codeword with Bitflip at the 4th position (compared to 7-bit vector codeword of example1 [1, 0, 1, 1, 0, 1, 0])
neg_ex1 = [1, 0, 1, 0, 0, 1, 0]
negative_example1 = np.array(neg_ex1)
parity_check(negative_example1)

The error vector is [0 0 1].
There was an error eoccuring upon code-transmission.
The Bitflip occured at the 4th position.


In [ ]:
# Artificial codeword with Bitflip at the 6th position (compared to 7-bit vector codeword of example2 [0, 0, 1, 1, 0, 0, 1])
neg_ex2 = [0, 0, 1, 1, 0, 1, 1]
negative_example2 = np.array(neg_ex2)
parity_check(negative_example2)

The error vector is [0 1 1].
There was an error eoccuring upon code-transmission.
The Bitflip occured at the 6th position.


In [ ]:
# Artificial codeword with Bitflip at the 1th position (compared to 7-bit vector codeword of example3 [0, 1, 1, 0, 0, 1, 1])
neg_ex3 = [1, 1, 1, 0, 0, 1, 1]
negative_example3 = np.array(neg_ex3)
parity_check(negative_example3)

The error vector is [1 0 0].
There was an error eoccuring upon code-transmission.
The Bitflip occured at the 1th position.


Examples for showcasing that partiy check functionality cannot check vectors with two-bit errors.

In [ ]:
# Artificial codeword with Bitflips at the 4th and 6th position (compared to 7-bit vector codeword of example1 [1, 0, 1, 1, 0, 1, 0])
extreme_neg_ex1 = [1, 0, 1, 0, 0, 0, 0]
extr_negative_example1 = np.array(extreme_neg_ex1)
parity_check(extr_negative_example1)

The error vector is [0 1 0].
There was an error eoccuring upon code-transmission.
The Bitflip occured at the 2th position.


Parity check function puts forward that the bitflip occured at the second position. This is not correct, since there were two bitflips occurring at the positions of the 4th and 6th elements (3rd and 5th index). This is, according to "Coding the Matrix" by Philip N. Klein Page 213, that there are multiple acceptable answers that satisfy the condition of H * e1 = H * e2.

In [24]:
ParityCheckMatrix.transpose()

array([[1, 0, 0],
       [0, 1, 0],
       [1, 1, 0],
       [0, 0, 1],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 1]])